### WBS Coding School
___
### --PROJECT--
# Solar Up: Solar Panel Object Detection
## Model training for image segmentation of solar panels

Use Google's GPU to run this script to substantially reduce training time.
(Next to "Connection" -> Click arrow -> "Change runtime type" -> choose a GPU, e.g. "T4 GPU")
___

#### Libraries

In [ ]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil

import ultralytics
from ultralytics import YOLO
from ultralytics import settings

from google.colab import files, drive

## 1.&nbsp; Download Data
We'll download the satellite images and a config yaml file from a Google Drive data dump.

The yaml tells the YOLOv8 model where to find the data and what the classes are.

In [ ]:
dataset_name = "kasmi_solar"

In [ ]:
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Download {dataset_name}_segmentation.yaml config file
# !gdown ...

# Download zipped training data folder
# !gdown https://drive.google.com/uc?id=1LoyR9ao8m7jSLBCGMBleB0d944XmGv0I

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1LoyR9ao8m7jSLBCGMBleB0d944XmGv0I 



Check hardware and directory structure. Make sure a GPU is being used for model training.

In [ ]:
ultralytics.checks()

Ultralytics YOLOv8.1.2 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [ ]:
# Display current path in Google Colab
%pwd

'/content'

In [ ]:
# Show directory contents
%ls -a

./  ../  1QFLKcoF93v-FEGEGXp60lMflwftdwji6  .config/


## 2. Model training

Here we'll load a pretrained YOLOv8 model (small, medium or large) and train it on the solar panel satellite data. Subsequently, we'll check its performance on the validation set.

In [ ]:
# Load a COCO-pretrained YOLOv8m model
model_name = 'yolov8l-seg'
model = YOLO(f'{model_name}.pt')

100%|██████████| 88.1M/88.1M [00:01<00:00, 58.3MB/s]


In [ ]:
# Display model information (optional)
model.info()

YOLOv8l-seg summary: 401 layers, 45997728 parameters, 0 gradients, 221.1 GFLOPs


(401, 45997728, 0, 221.13428480000005)

In [ ]:
# Train the model on our custom dataset
config_file = f'{dataset_name}_segmentation.yaml'
epochs = 50
img_size = 400
experiment = f'{model_name}_{epochs}'
device = 0 # 0 = cuda, i.e. the GPU device; "cpu" = cpu

results = model.train(data=config_file, epochs=epochs, imgsz=img_size, name=experiment, device=device)

## 3. Evaluate Model

In [ ]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered

metrics.box.map    # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps   # a list contains map50-95(B) of each category
metrics.seg.map    # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps   # a list contains map50-95(M) of each category

## 4. Export Model Weights

In [ ]:
# Load a model
path_to_best = f"runs/detect/{experiment}/weights/best.pt" # .pt = PyTorch model
model = YOLO(path_to_best)  # load a custom trained model

# Export the model
model.export() # the default format is 'torchscript' (PyTorch)

In [ ]:
# Specify the output path for the ZIP archive
output_path = f"models/trained_model_{experiment}"
print(output_path)

shutil.make_archive(output_path, 'zip', f"/content/runs/detect/{experiment}")

In [ ]:
# Download model weights:
files.download(f"runs/detect/{experiment}/weights/best.torchscript")

In [ ]:
files.download(f"models/trained_model_{experiment}.zip")